#  天池新人实战赛o2o优惠券使用预测

> 

In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
import os, sys, pickle
%matplotlib inline

In [2]:
# 读取数据
df_offline = pd.read_csv("../ali-data/ccf_offline_stage1_train.csv")
#df_online = pd.read_csv("./data/ccf_online_stage1_train.csv")
df_test = pd.read_csv("../ali-data/ccf_offline_stage1_test_revised.csv")

In [3]:
# offline 文件
df_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


    User_id        用户id

    Merchant_id     商家id

    Coupon_id	    优惠券id

    Discount_rate	  满x减y

    Distance	    x*500米,user常活动的地点与店铺距离，0：小于500米，10：大于5公里 

    Date_received  	领劵日期

    Date          消费日期

* 可能用的到的特征是“Discount_rate”“Distance”* 


In [33]:
# # 去掉缺失值22-
# df_offline_drop_miss = df_offline[["Discount_rate", "Distance"]].dropna()
# #df_offline_drop_miss = df_offline.dropna()
# #histogram
# sns.distplot(df_offline_drop_miss['Distance'])
# #

## 处理标签

"Date" 就是我们要预测的结果，但不是预测日期，而是有没有使用。

 不同情况打上不同标签：
 
- 收到优惠卷，未消费： -1
- 未收到优惠卷，15天内消费： 0
- 收到优惠卷， 消费：  1

In [59]:
# 合并train 和test，同时做处理
data = pd.concat([df_offline, df_test], keys=(["train", "test"]))

#data["Date"] = pd.to_datetime(date["Date"],format="%Y%m%d")
#data["Date_received"] = pd.to_datetime(date["Date_received"],format="%Y%m%d")

day15 = pd.Timedelta(15,"D")
def makeLabel(row):
    if pd.isnull(row["Date_received"]) : # and pd.isna(row["Date"]) == False:
        return -1
    elif row["Date"] != "null" :
        res = pd.to_datetime(row["Date"],format="%Y%m%d") - pd.to_datetime(row["Date_received"],format="%Y%m%d")
        if res <= day15:
            return 1
    return 0

data["label"] = data[:].apply(makeLabel, axis=1)
data.head()
data["label"].unique()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


array([-1,  0,  1], dtype=int64)

## 保存一下

In [60]:
data.to_csv("../ali-data/feature.csv")

In [61]:
data = pd.read_csv("../ali-data/feature.csv")

In [62]:
data.head()

,Unnamed: 0,Unnamed: 1,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id,label
0,train,0,NaN,20160217.0,NaN,NaN,0.0,2632,1439408,-1
1,train,1,11002.0,NaN,20160528.0,150:20,1.0,4663,1439408,0
2,train,2,8591.0,NaN,20160217.0,20:1,0.0,2632,1439408,0
3,train,3,1078.0,NaN,20160319.0,20:1,0.0,2632,1439408,0
4,train,4,8591.0,NaN,20160613.0,20:1,0.0,2632,1439408,0


## 特征

### 商户相关特征

### 1. 折扣率

-  "Discount_rate"   满x减y
-  用折扣率 1-y/x代替

In [6]:
## 特征1: 折扣率

def rate(x):
    x = str(x)
    length = len(x)
    a=1
    b=1
    try:
        if length == 3:
            return 1.0
        elif ":" in x:
            a,b = x.split(":")
            return 1-float(b)/float(a)
    except:
        return x

data["f_discount_rate"] = data["Discount_rate"].apply(lambda x : rate(x))



### 2. 折扣类型

-  1 满减
-  2 折扣率
-  3 无折扣

* 然后  ont-hot 编码

In [7]:
def discount_cls(x):
    x = str(x)
    length = len(x)
    if length == 3:
        return 3
    elif ":" in x:
        return 1
    else:
        return 2

data["f_discount_cls"] = data["Discount_rate"].copy()
data["f_discount_cls"] = data["f_discount_cls"].apply(lambda x : discount_cls(x))

# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(categorical_features=[0])
# ohe.fit_transform(data["f_discount_cls"])
data[["f_cls_man_jian", "f_cls_discount","f_cls_None"]]=pd.get_dummies(data["f_discount_cls"])

data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id,label,f_discount_rate,f_discount_cls,f_cls_man_jian,f_cls_discount,f_cls_None,f_activity_intensity
0,0,train,0,NaN,20160217.0,NaN,NaN,0.0,2632,1439408,-1,1.000000,3,0,0,1,3
1,1,train,1,11002.0,NaN,20160528.0,150:20,1.0,4663,1439408,0,0.866667,1,1,0,0,1
2,2,train,2,8591.0,NaN,20160217.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1
3,3,train,3,1078.0,NaN,20160319.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1
4,4,train,4,8591.0,NaN,20160613.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1


### 3. 活动力度

分级：

- 无活动      0
    
- 0～10      1
    
- 10～100    2
    
- 100～1000  3
    
- 折扣率      3

In [8]:

def Activity_intensity(x):
    
    if pd.isnull(x):
        return 0
    x = str(x)
    length = len(x)
    a=1
    b=1

    if ":" in x:
        a,b = x.split(":")
        b=int(b)
        
        if b < 10:
            return 1.0
        elif b < 100:
            return 2.0
        return 3.0
    else:
        return 3.0
        
    
data["f_activity_intensity_x"] = data["Discount_rate"].apply(lambda x : Activity_intensity(x))
data["f_activity_intensity_x"].unique()

array([0., 2., 1., 3.])

## 消费者

### 1. 购物次数


In [9]:
temp = user_coupon_consume.size().reset_index(name='u2')
X = pd.merge(X, temp, how='left', on='User_id')


user_id_data = data[data["Date"] != "null"][["User_id"]].copy()

user_id_data["f_count"] = 1

User = user_id_data.groupby(["User_id"], as_index = False ).count()

data2 = pd.merge(data, User, on = "User_id", how = "left")
data2.head()



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id,label,f_discount_rate,f_discount_cls,f_cls_man_jian,f_cls_discount,f_cls_None,f_activity_intensity,f_activity_intensity_x,f_count
0,0,train,0,NaN,20160217.0,NaN,NaN,0.0,2632,1439408,-1,1.000000,3,0,0,1,3,0.0,8
1,1,train,1,11002.0,NaN,20160528.0,150:20,1.0,4663,1439408,0,0.866667,1,1,0,0,1,2.0,8
2,2,train,2,8591.0,NaN,20160217.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1,1.0,8
3,3,train,3,1078.0,NaN,20160319.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1,1.0,8
4,4,train,4,8591.0,NaN,20160613.0,20:1,0.0,2632,1439408,0,0.950000,1,1,0,0,1,1.0,8


### 2. 优惠卷使用率

In [10]:
data2["label"].value_counts()

 0    1103498
-1     701602
 1      63424
Name: label, dtype: int64

### 3.

## 拆开数据, 选择特征

In [58]:
# 拆开数据
# train_data=data.xs('train')
# test_data=data.xs('test')#.drop(labels='label',axis=1)
train_data = data2[data2.iloc[:,1]== "train"]
train_data = train_data.iloc[:,3:]

test_data = data2[data2.iloc[:,1]== "test"]
test_data = test_data.iloc[:,3:]


train_data = train_data.drop(["Date","Date_received","Discount_rate","Coupon_id",
                              "Merchant_id","User_id",
                             ], axis=1)
                           
train_data = train_data.fillna(0)
test_data = test_data.drop(["Date","Date_received","Discount_rate","Coupon_id",
                            "Merchant_id","User_id",
                           ], axis=1)

x_test = test_data.fillna(0)

# 打乱数据
# train_data = train_data.sample(frac=1.0)
# test_data = x_test.sample(frac=1.0)
train_data.head()

,Distance,label,f_discount_rate,f_discount_cls,f_cls_man_jian,f_cls_discount,f_cls_None,f_activity_intensity,f_activity_intensity_x,f_count
0,0.0,-1,1.000000,3,0,0,1,3,0.0,8
1,1.0,0,0.866667,1,1,0,0,1,2.0,8
2,0.0,0,0.950000,1,1,0,0,1,1.0,8
3,0.0,0,0.950000,1,1,0,0,1,1.0,8
4,0.0,0,0.950000,1,1,0,0,1,1.0,8


In [49]:
train_data.sort_values(["label"],inplace=True)

train_data = train_data[900000:]
train_data["label"].value_counts()

0    89858
1    63424
Name: label, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
# train_data.head()


## 删除  普通消费
train_data = train_data[(train_data["label"] != -1)]
print(train_data["label"].unique())

train_label = train_data["label"]
train_data = train_data.drop("label", axis=1)


train_data,val_data, train_label ,val_label = train_test_split(train_data,train_label,test_size=0.3,random_state=6)

print("划分后")
print("train %d : 验证 %d"%(len(train_label),len(val_label)))

[0 1]
划分后
train 107297 : 验证 45985


In [51]:
tmp = pd.DataFrame(train_label)

tmp["label"].value_counts()

0    62929
1    44368
Name: label, dtype: int64

## 划分训练集，验证集

In [53]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

S=StandardScaler()

S.fit(train_data)

x_train_stand = S.transform(train_data)

x_val_stand = S.transform(val_data)


print(x_train_stand[1])
x_test_stand=S.transform(test_data.drop(["label"],axis=1))


print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(x_train_stand, train_label)
print("end")
# fpr, tpr, thresholds = roc_curve(val_label, prediction[:,1], pos_label=1)
# print("auc:",auc(fpr, tpr))


# auc_score1 = myAUC(val_label,prediction[:,0])

# print(Log.score(x_val_stand,val_label))
# print(auc_score1)

# result=pd.DataFrame({'PassengerId':test_data.index,'Survived':res[:,0]})  #这里需要注意把prediction的数据转换成Int型不然系统判定不了，得分会为0 
# result.to_csv('../ali-data/result.csv',index=False) #设置不输出Index

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have bee

[ 0.25727683  0.49499505 -0.21018675  0.22320716 -0.16971584 -0.14087658
 -0.21018675  0.99490777 -0.42008973]
----train-----
end


In [54]:
# #### 预测以及结果评价
print(model.score(x_val_stand, val_label))

# val = val_data.copy()
# val['pred_prob'] = model.predict_proba(x_val_stand)[:,1]
# validgroup = val.groupby(['Coupon_id'])
# aucs = []
# for i in validgroup:
#     tmpdf = i[1] 
#     if len(tmpdf['label'].unique()) != 2:
#         continue
#     fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
#     aucs.append(auc(fpr, tpr))
# print(np.average(aucs))

0.7316081330868761


In [ ]:
#pre = model.predict_proba(x_test_stand)


In [57]:
# test prediction for submission
y_test_pred = model.predict_proba(x_test_stand)
dftest1 = df_test[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('../ali-data/submit1.csv', index=False, header=False)
dftest1.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,20160712,0.189898
1,6949378,3429,20160706,0.165072
2,2166529,6928,20160727,0.467966
3,2166529,1808,20160727,0.493772
4,6172162,6500,20160708,0.199331


In [56]:

dftest1 = df_test[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = res[:,1]
dftest1.to_csv('../ali-data/submit1.csv', index=False, header=False)
dftest1.head(10)

# avgAUC calculation
vg = dftest1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

NameError: name 'res' is not defined